# Introduction to Tensor

In [ ]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

In [ ]:
### Her bir tensörün veri tipi ve şekli vardır.
## Veri tipleri: float32, int32, string, others.
## Şekli: Basitce verinin boyutu denebilir.

# Farklı tensör çeşitlerinin oluşturulmasına örnekler

string = tf.Variable('Bu bir dizidir.', tf.string)
number = tf.Variable(297, tf.int16)
flaot = tf.Variable(3.16, tf.float64)

In [ ]:
### Rank/Degree Tensörde yer alan boyutların sayısı anlamına gelir.

print('string değişkeninin boyutu: ', tf.rank(string)) ## 0 boyutlu yani skaler değişken.

rank1_tensor = tf.Variable(['Test'], tf.string)
rank2_tensor = tf.Variable([['test','ok'],['test','yes']], tf.string)

print('rank1_tensor boyutu: ', tf.rank(rank1_tensor)) ## 1 boyutlu yani vektör.
print('rank_2tensor boyutu: ', tf.rank(rank2_tensor)) ## 2 boyutlu yani matrix.

In [ ]:
### Shape tensör içinde her boyutta bize kaç eleman olduğunu söyler.

print('number değişkeninin şekli(shape)', number.shape) ## Sıfır boyutlu yani skaler
print('rank1_tensörünün şekli(shape)', rank1_tensor.shape) ## Bir boyutlu bir elemanlı
print('rank2_tensörünün şekli(shape)', rank2_tensor.shape) ## İki boyutlu iki elemanlı (Liste içersinde iki liste her listede 2 eleman var.1)

In [ ]:
### Tensörlerde Şekil(Reshape) değiştirme

tensor1 = tf.ones([1, 2, 3]) ## 1 lerden oluşan bir tensör yani rank = 3, shape = [1, 2, 3] (Liste içersinde bir liste o listenin içersinde iki liste her listede 3 eleman)
tensor2 = tf.reshape(tensor1, [2, 3, 1]) ## (Liste içersinde iki liste her iki liste içersinde 3 liste her 3 üç liste içersinde 1 eleman) 
tensor3 = tf.reshape(tensor2, [3, -1]) ## (Liste içersinde 3 liste bu listelerin içinde 2 şer eleman) tensor2 toplam 6 elemanlı bu elemanları 3 listeye 2 şer dağıtılmış hali 
## Tensor 3 deki -1 her listnin içersinde kaç eleman olduğu hesaplar yani toplam 6 elemanlı tensor2 yi tensor3 e çevirirken 6 elemanı 2 şerli olarak 3 listeye sığdıraçağını

In [ ]:
### Tensör çeşitleri
## En çok kullanılan tensör çeşitleri:
# Variable
# Constant
# Placeholder
# SparseTensor

In [ ]:
### Example

tensor = tf.zeros([5,5,5,5]) ## 1 listenin içerisinde 5 liste bu 5 listenin her birinin içerisinde 5 er liste bu listelerin içersinde 5 er liste bu 5 er listelerin içersinde 5 er eleman
## Toplamda 5 * 5 * 5 * 5 = 625 eleman
tensor = tf.reshape(tensor, [625]) ## Bir nevi flatten işlemi 625 elemanlı bir liste yani vektor 1 boyutlu
tensor = tf.reshape(tensor, [125, -1]) ## Bir liste içerisinde 125 liste her liste içersinde 625 / 125 tane eleman
tensor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from six.moves import urllib
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc

In [ ]:
df_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
df_test = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

y_train = df_train.pop('survived')
y_eval = df_test.pop('survived')

In [ ]:
print(df_train.loc[0], y_train.loc[0]) ## .loc 0.satır

In [ ]:
df_train.head()

In [ ]:
print(df_train.describe(), end='\n\n')
print('mean of age: ', df_train.describe().mean().age) ## OR df_train.describe().mean['age]

In [ ]:
df_train.shape ## 627 satır ve 9 sütündan oluşan bir dataframe

In [ ]:
df_train.age.hist(bins= 20)

In [ ]:
df_train.sex.value_counts().plot(kind= 'bar')

In [ ]:
df_train['class'].value_counts().plot(kind= 'bar')

In [ ]:
pd.concat([df_train, y_train], axis= 1).groupby('sex')['survived'].mean().plot(kind= 'bar').set_xlabel(r'% survive')

In [ ]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

feature_columns = []

for feature_name, Columns in df_train.items():
    if is_string_dtype(Columns):
        vocabulary = df_train[feature_name].unique()
        feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    else:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [ ]:
def make_input_fn(df, label, num_epochs= 10, shuffle= True, batch_size= 32):
    def input_func():
        ds = tf.data.Dataset.from_tensor_slices((dict(df), label))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_func        

In [ ]:
df_train_input_fn = make_input_fn(df_train, y_train)
df_test_input_fn = make_input_fn(df_test, y_eval, num_epochs= 1, shuffle= False)

In [ ]:
for feature_batch, label_batch in df_train_input_fn().take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['sex'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

In [ ]:
age_column = feature_columns[1]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()  ## tf.keras.layers.DenseFeatures katmanını kullanarak belirli bir özellik sütununun sonucunu inceleyebilirsiniz

In [ ]:
sex_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(sex_column)])(feature_batch).numpy() ## DenseFeatures yalnızca yoğun tensörleri kabul eder, kategorik bir sütunu incelemek için önce bunu bir gösterge sütununa dönüştürmeniz gerekir:

In [ ]:
from IPython.display import clear_output

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(df_train_input_fn)
result = linear_est.evaluate(df_test_input_fn)

clear_output()
print(result)

In [ ]:
result = list(linear_est.predict(df_test_input_fn))
clear_output()
print(df_test.loc[0], y_eval[0])
print(result[0]['probabilities'][1])

In [ ]:
probs = pd.Series([pred['probabilities'][1] for pred in result])
probs

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)

In [58]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [59]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [60]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [61]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [62]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [63]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [64]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Madao\\AppData\\Local\\Temp\\tmpmx8bac6m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [65]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Madao\AppData\Local\Temp\tmpmx8bac6m\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.3938713, step = 0
INFO:tensorflow:global_step/sec: 561.67
INFO:tensorflow:loss = 1.0097805, step = 100 (0.178 sec)
INFO:tensorflow:global_step/sec: 769.057
INFO:tensorflow:loss = 0.9494438, step = 200 (0.130 sec)
INFO:tensorflow:global_step/sec: 799.815
INFO:tensorflow:loss = 0.9166491, step = 300 (0.126 sec)
INFO:tensorflow:global_step/sec: 781.083
INFO:tensorflow:loss = 0.8817495, step = 400 (0.127 sec)
INFO:tensorflow:global_step/sec: 826.252
INFO:tensorflow:loss = 0.8602422, s

In [66]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-09-04T22:06:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Madao\AppData\Local\Temp\tmpmx8bac6m\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18504s
INFO:tensorflow:Finished evaluation at 2021-09-04-22:06:21
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.73333335, average_loss = 0.5436447, global_step = 5000, loss = 0.5436447
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\Madao\AppData\Local\Temp\tmpmx8bac6m\model.ckpt-5000

Test set accuracy: 0.733



In [67]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [68]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Madao\AppData\Local\Temp\tmpmx8bac6m\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (81.2%), expected "Setosa"
Prediction is "Versicolor" (45.6%), expected "Versicolor"
Prediction is "Virginica" (60.1%), expected "Virginica"
